# Imports

In [1]:
import sys
import mrob
import time
import os
from octreelib.grid import Grid, GridConfig, VisualizationConfig
from octreelib.octree import MultiPoseOctree, OctreeConfig
from typing import Tuple, List
from dataclasses import dataclass
import numpy as np
import open3d as o3d

sys.path.append("..")
from slam.backend import BaregBackend, EigenFactorBackend, Backend, BackendOutput
from slam.pipeline import StaticPipeline, StaticPipelineRuntimeParameters
from slam.segmenter import Segmenter, CAPESegmenter, RansacSegmenter
from slam.subdivider import Subdivider, CountSubdivider, EigenValueSubdivider, SizeSubdivider
from slam.utils import Reader, HiltiReader, KittiReader

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Benchmark configuration

In [2]:
SAMPLES_COUNT = 10

### Utils

In [3]:
def evaluate(timestamps: List[float]):
    """
    Evaluates given timestamps List by calculating min, max, mean and std values
    """
    def print_metrics(ts):
        print(f"\tmin = {round(np.min(ts), 3)}s")
        print(f"\tmax = {round(np.max(ts), 3)}s")
        print(f"\tmean = {round(np.mean(ts), 3)}s")
        print(f"\tstd = {round(np.std(ts), 3)}s")
        
    timestamps = np.array(timestamps)
    print_metrics(timestamps)

def read_patch(reader: Reader,path: str, start: int, end: int) -> List[o3d.geometry.PointCloud]:
    """
    Reads patch of point clouds
    """
    point_clouds = []
    
    hilti_clouds = os.path.join(path, "clouds")
    hilti_poses = os.path.join(path, "poses")
    
    for ind in range(start, end):
        point_cloud_path = os.path.join(hilti_clouds, str(ind) + ".pcd")
        pose_path = os.path.join(hilti_poses, str(ind) + ".txt")
        
        point_cloud = reader.read_point_cloud(filename=point_cloud_path)
        pose = reader.read_pose(filename=pose_path)

        point_clouds.append(point_cloud.transform(pose))

    return point_clouds

# Pipeline configuration

In [4]:
@dataclass
class PipelineConfiguration:
    """
    Represents pipeline configuration
    """
    subdividers: List[Subdivider]
    segmenters: List[Segmenter]
    backend: Backend
    grid: Grid


@dataclass
class PipelineDurations:
    """
    Represents duration of each pipeline stage
    """
    initialization: float
    subdividers: float
    distribution: float
    segmenters: float
    backend: float


def run_pipeline(point_clouds: List[o3d.geometry.PointCloud], pipeline: PipelineConfiguration) -> PipelineDurations:
    """
    Runs pipeline with time benchmarking
    """
    
    middle_pose_number = len(point_clouds) // 2

    initialization_start = time.perf_counter()
    pipeline.grid.insert_points(
            middle_pose_number,
            point_clouds[middle_pose_number].points,
    )
    initialization_end = time.perf_counter() - initialization_start
    
    subdividers_start = time.perf_counter()
    pipeline.grid.subdivide(pipeline.subdividers)
    subdividers_end = time.perf_counter() - subdividers_start

    distribution_start = time.perf_counter()
    for pose_number, point_cloud in enumerate(point_clouds):
        if pose_number == middle_pose_number:
            continue
        pipeline.grid.insert_points(pose_number, point_cloud.points)
    distribution_end = time.perf_counter() - distribution_start

    segmenters_start = time.perf_counter()
    for segmenter in pipeline.segmenters:
        pipeline.grid.map_leaf_points(segmenter)
    segmenters_end = time.perf_counter() - segmenters_start

    backend_start = time.perf_counter()
    pipeline.backend.process(pipeline.grid)
    backend_end = time.perf_counter() - backend_start

    return PipelineDurations(
        initialization=initialization_end,
        subdividers=subdividers_end,
        distribution=distribution_end,
        segmenters=segmenters_end,
        backend=backend_end,
    )

# Evaluation

In [5]:
# Pipeline configuration
# TODO(user): You can manipulate configuration spec below as you want
dataset_path = "../evaluation/kitti"
dataset_reader = KittiReader
start = 0
end = 3
step = 3
initial_voxel_size = 4

def create_configuration() -> PipelineConfiguration:
    subdividers = [
        SizeSubdivider(
            size=2,
        ),
    ]
    
    segmenters = [
        RansacSegmenter(
            threshold=0.01,
            initial_points=6,
            iterations=5000,
        ),
        CAPESegmenter(
            correlation=100,
        )
    ]
        
    backend = EigenFactorBackend(
        poses_number=step,
        iterations_number=5000,
    )

    grid = Grid(
        GridConfig(
            octree_type=MultiPoseOctree,
            octree_config=OctreeConfig(),
            grid_voxel_edge_length=4,
        )
    )
        
    return PipelineConfiguration(
        subdividers=subdividers,
        segmenters=segmenters,
        backend=backend,
        grid=grid,
    )
# Do not touch code below, just run it :)

for ind in range(start, end, step):
    print(f"Patch {ind} -> {ind + step}; Samples count = {SAMPLES_COUNT}")
    
    initialization_timestamps = []
    subdividers_timestamps = []
    distribution_timestamps = []
    segmenters_timestamps = []
    backend_timestamps = []
    
    for sample in range(SAMPLES_COUNT):
        point_clouds = read_patch(dataset_reader, dataset_path, ind, ind + step)
        
        pipeline_durations = run_pipeline(point_clouds, create_configuration())
        
        initialization_timestamps.append(pipeline_durations.initialization)
        subdividers_timestamps.append(pipeline_durations.subdividers)
        distribution_timestamps.append(pipeline_durations.distribution)
        segmenters_timestamps.append(pipeline_durations.segmenters)
        backend_timestamps.append(pipeline_durations.backend)

    print("First point clouds insertion stage")
    evaluate(initialization_timestamps)
    print("Subdividers stage")
    evaluate(subdividers_timestamps)
    print("Distribution stage")
    evaluate(distribution_timestamps)
    print("Segmenters stage")
    evaluate(segmenters_timestamps)
    print("Backend stage")
    evaluate(backend_timestamps)

Patch 0 -> 3; Samples count = 10
First point clouds insertion stage
	min = 2.647s
	max = 2.863s
	mean = 2.708s
	std = 0.066s
Subdividers stage
	min = 8.991s
	max = 9.287s
	mean = 9.077s
	std = 0.087s
Distribution stage
	min = 20.751s
	max = 21.297s
	mean = 20.877s
	std = 0.151s
Segmenters stage
	min = 7.337s
	max = 9.898s
	mean = 7.762s
	std = 0.769s
Backend stage
	min = 0.096s
	max = 0.155s
	mean = 0.104s
	std = 0.017s
